# EDA - Rais

**E**xploratory **D**ata **A**nalysis on RAIS Database - Florianópolis, SC - Brasil

**Authors:**
- Luis Felipe Pelison
- Fernando Battisti
- Ígor Yamamoto

Dictionary: 
ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/Layouts/v%C3%ADnculos/RAIS_vinculos_layout2018.xls

# Imports

In [54]:
import pandas as pd
import numpy as np

pd.set_option('max_rows', 200)

# Open Data

In [7]:
df = pd.read_parquet('dados/rais_floripa_2018.parquet')
print(df.shape)
df.head(2)

(432486, 16)


,CNAE 2.0 Subclasse,Escolaridade após 2005,Idade,Mês Admissão,Mês Desligamento,Motivo Desligamento,Município,Qtd Hora Contr,Raça Cor,Sexo Trabalhador,Tamanho Estabelecimento,Tempo Emprego,Tipo Defic,Vl Remun Média Nom,UF,CBO 2002
4470590,8112500,1,23,0,3,12,420540,44,9,1,4,"2,9",0,"1.484,79",42,514320
4470599,7112000,1,33,3,4,11,420540,44,9,1,3,"1,9",0,"1.290,00",42,717020


# Pre Processing

In [8]:
CAT_FEATURES = ['CNAE 2.0 Subclasse', 'Escolaridade após 2005', 'Mês Admissão', 'Mês Desligamento', 'Motivo Desligamento', 'Município', 'Raça Cor', 'Sexo Trabalhador', 'Tamanho Estabelecimento', 'Tipo Defic', 'UF', 'CBO 2002']

for cat_feat in CAT_FEATURES:
    df[cat_feat] = df[cat_feat].astype('str')

df['Tempo Emprego'] = df['Tempo Emprego'].str.replace(',','.').astype('float')

df['Vl Remun Média Nom'] = df['Vl Remun Média Nom'].str.replace('.', '').str.replace(',','.').astype('float')

## Tamanho Estabelecimento

In [9]:
df['Tamanho Estabelecimento'].value_counts()

10    174502
5      40351
4      36555
7      34376
3      32859
2      31635
6      25907
9      25797
8      22098
1       8406
Name: Tamanho Estabelecimento, dtype: int64

In [17]:
df['Tamanho Estabelecimento'] = (
    df['Tamanho Estabelecimento']
    .map(
        {
            '1': 'ZERO',
            '2': 'ATE_4',
            '3': 'DE_5_A_9',
            '4': 'DE_10_A_19',
            '5': 'DE_20_A_49',
            '6': 'DE_50_A_99',
            '7': 'DE_100_A_249',
            '8': 'DE_250_A_499',
            '9': 'DE_500_A_999',
            '10': '1000_OU_MAIS',
            '-1': 'IGNORADO',
        }
    )
)

In [19]:
df['Tamanho Estabelecimento'].value_counts()

1000_OU_MAIS    174502
DE_20_A_49       40351
DE_10_A_19       36555
DE_100_A_249     34376
DE_5_A_9         32859
ATE_4            31635
DE_50_A_99       25907
DE_500_A_999     25797
DE_250_A_499     22098
ZERO              8406
Name: Tamanho Estabelecimento, dtype: int64

## CBO

In [95]:
df['CBO 2002'] = df['CBO 2002'].apply(lambda x: 'Unknown' if x == '0000-1' else x)

# Analysis

In [20]:
# All from Florianopolis

df['Município'].value_counts()

420540    432486
Name: Município, dtype: int64

In [97]:
# List Categories

for col in df.columns:
    col_type = df[col].dtype
    print(f'     ***Type: {col_type}')
    if ((col_type == 'O') or (hasattr(df[col], 'cat'))):
        print(f'     Top 5 of Column') #: {col}')
        display(df[col].value_counts(normalize=True)[:5]*100)
        print('-----')
        print('\n')
    else:
        print(f'     Describe Column: {col}')
        display(df[col].describe())
        print('-----')
        print('\n')

     ***Type: object
     Top 5 of Column


8411600    23.354513
8424800     5.244332
7830200     3.971689
5611201     3.632719
8423000     2.753615
Name: CNAE 2.0 Subclasse, dtype: float64

-----


     ***Type: object
     Top 5 of Column


9    43.819916
7    36.274238
5     5.021434
8     4.925940
6     3.917121
Name: Escolaridade após 2005, dtype: float64

-----


     ***Type: int64
     Describe Column: Idade


...

-----


     ***Type: object
     Top 5 of Column


42    100.0
Name: UF, dtype: float64

-----


     ***Type: object
     Top 5 of Column


331205    12.904926
411010     6.352576
514320     4.263028
521110     4.057010
411005     3.034549
Name: CBO 2002, dtype: float64

-----




In [98]:
df[df['Qtd Hora Contr'] == 1]['CBO 2002'].value_counts()[:4]

513405    109
421125     88
422105     72
234520     57
Name: CBO 2002, dtype: int64

**513405**: Garçom

**421125**: Operador de Caixa

**422105**: Recepcionista Atendente

**234520**: Prof. Ensino Superior

In [99]:
df[df['Qtd Hora Contr'] == 1]['CNAE 2.0 Subclasse'].value_counts()[:4]

5611204    132
5611201     91
4711302     90
5223100     62
Name: CNAE 2.0 Subclasse, dtype: int64

**5611204**: Bares e outros estabelecimentos especializados em servir bebidas, sem entretenimento

**5611201**: Restaurantes e similares

**4711302**: Comércio varejista de mercadorias em geral, com predominância de produtos alimentícios - supermercados

**5223100**: Estacionamento de veículos

# Challenge

In [31]:
df.head()

,CNAE 2.0 Subclasse,Escolaridade após 2005,Idade,Mês Admissão,Mês Desligamento,Motivo Desligamento,Município,Qtd Hora Contr,Raça Cor,Sexo Trabalhador,Tamanho Estabelecimento,Tempo Emprego,Tipo Defic,Vl Remun Média Nom,UF,CBO 2002
4470590,8112500,1,23,0,3,12,420540,44,9,1,DE_10_A_19,2.9,0,1484.79,42,514320
4470599,7112000,1,33,3,4,11,420540,44,9,1,DE_5_A_9,1.9,0,1290.00,42,717020
4470606,5611201,1,36,2,3,12,420540,22,2,1,DE_20_A_49,1.3,0,951.63,42,513405
4470607,5611201,1,27,0,1,12,420540,9,2,1,DE_20_A_49,1.9,0,664.41,42,513405
4470608,4110700,1,29,0,3,11,420540,44,9,1,DE_5_A_9,38.7,0,1841.74,42,715210


In [34]:
df.median()

CNAE 2.0 Subclasse        8121400.000
Escolaridade após 2005          8.000
Idade                          36.000
Mês Admissão                    0.000
Mês Desligamento                0.000
Motivo Desligamento             0.000
Município                  420540.000
Qtd Hora Contr                 40.000
Raça Cor                        8.000
Sexo Trabalhador                2.000
Tempo Emprego                  18.700
Tipo Defic                      0.000
Vl Remun Média Nom           2104.875
UF                             42.000
dtype: float64

In [102]:
## Add Escolaridade, Cnae...


def report_salary(df: pd.DataFrame, idade: int, cbo: str, raca: str, genero: str, threshold: int = 1) -> None:
    df_filtered = df[
        (df['Idade'] == idade) &
        (df['CBO 2002'] == cbo) &
        (df['Raça Cor'] == raca) &
        (df['Sexo Trabalhador'] == genero)
    ]
    
    row_count = df_filtered.shape[0]
    print(f'[debug] Rows after filter: {row_count}')

    # The filter returned few rows, so give it a second try!
    if row_count < threshold:
        print(f'[debug] The rows after filter are less than threshold!')
        df_filtered = _similar_cbo(df, idade, cbo, raca, genero)
        row_count = df_filtered.shape[0]
        print(f'[debug] Rows after second try: {row_count}')
    
    _print_report(df_filtered, 'Vl Remun Média Nom')
        
    return None


def _similar_cbo(df: pd.DataFrame, idade: int, cbo: str, raca: int, genero: int) -> pd.DataFrame:
    """Finds another CBO based on similarity.
    
    """
    # Filters without CBO
    df_filtered_no_cbo = df[
        (df['Idade'] == idade)
        & (df['Raça Cor'] == raca)
        & (df['Sexo Trabalhador'] == genero)
    ]
    
    # Get all CBOs with the filter above
    list_cbos = df_filtered_no_cbo['CBO 2002'].unique()
    
    # For all CBOs, find the most similar to original CBO
    cbo = int(cbo)
    closer_cbo = ''
    closer_cbo_score = 9999999

    for cbos in list_cbos:
        try:
            int_cbos = int(cbos)
        except:
            int_cbos = 9999999999
        diff_cbos = np.abs(int_cbos - cbo)
        if diff_cbos < closer_cbo_score:
            closer_cbo = cbos
            closer_cbo_score = diff_cbos
            print(f'[debug] Closer CBO: {cbos}')
            print(f'[debug] Closer CBO score: {closer_cbo_score}')

    # Return the data with all filters including new similar CBO together
    return df_filtered_no_cbo[df_filtered_no_cbo['CBO 2002'] == closer_cbo]


def _print_report(df: pd.DataFrame, col_print: str) -> None:
    print('\n')
    print('----- RESULTS -----')
    print(f"Median\t {df[col_print].median()}")
    print(f"Mean\t {df[col_print].mean()}")
    
    return None

In [82]:
report_salary(df, 23, '212205', '2', '1') # 212205

[debug] Rows after filter: 0
[debug] The rows after filter are less than threshold!
[debug] Closer CBO: 724205
[debug] Closer CBO score: 512000
[debug] Closer CBO: 514325
[debug] Closer CBO score: 302120
[debug] Closer CBO: 513505
[debug] Closer CBO score: 301300
[debug] Closer CBO: 421125
[debug] Closer CBO score: 208920
[debug] Closer CBO: 414105
[debug] Closer CBO score: 201900
[debug] Closer CBO: 373205
[debug] Closer CBO score: 161000
[debug] Closer CBO: 141515
[debug] Closer CBO score: 70690
[debug] Closer CBO: 142320
[debug] Closer CBO score: 69885
[debug] Closer CBO: 233215
[debug] Closer CBO score: 21010
[debug] Closer CBO: 212315
[debug] Closer CBO score: 110
[debug] Closer CBO: 212310
[debug] Closer CBO score: 105
[debug] Closer CBO: 212305
[debug] Closer CBO score: 100
[debug] Rows after second try: 1


----- RESULTS -----
Median	 4221.32
Mean	 4221.32


In [103]:
report_salary(df, 40, '211205', '2', '1') # 212205

[debug] Rows after filter: 0
[debug] The rows after filter are less than threshold!
[debug] Closer CBO: 771105
[debug] Closer CBO score: 559900
[debug] Closer CBO: 513205
[debug] Closer CBO score: 302000
[debug] Closer CBO: 510310
[debug] Closer CBO score: 299105
[debug] Closer CBO: 341225
[debug] Closer CBO score: 130020
[debug] Closer CBO: 331110
[debug] Closer CBO score: 119905
[debug] Closer CBO: 317210
[debug] Closer CBO score: 106005
[debug] Closer CBO: 141510
[debug] Closer CBO score: 69695
[debug] Closer CBO: 261715
[debug] Closer CBO score: 50510
[debug] Closer CBO: 224120
[debug] Closer CBO score: 12915
[debug] Closer CBO: 212405
[debug] Closer CBO score: 1200
[debug] Closer CBO: 212215
[debug] Closer CBO score: 1010
[debug] Closer CBO: 211105
[debug] Closer CBO score: 100
[debug] Closer CBO: 211110
[debug] Closer CBO score: 95
[debug] Rows after second try: 2


----- RESULTS -----
Median	 12187.115
Mean	 12187.115


# Ideias

- Fazer media entre os 3 CBOS mais similares
- Mostrar gráfico de tendencia de salario mediano para idades superiores (+1, +2, +3, ... +10 anos da idade passada)

# The End